In [97]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import re

In [98]:
file = "data/raw_logs/final_server_logs12.csv"
df = pd.read_csv(file,on_bad_lines='skip')

In [99]:
df.head()

,Timestamp,IP Address,Session ID,Country,Method,URL,Status Code,Response Time (ms),Sales Agent,Referrer,Product,Price,IP_Session,viewed_pricing_after_demo,pages_after_demo,sessions_after_demo,time_to_purchase
0,2024-11-27 02:51:02,155.200.68.16,9ac6b5fa-b78b-4a95-a550-0621f2173d99,USA,GET,/about-us.html,404,241.91,NaN,https://www.linkedin.com,NaN,0.0,155.200.68.16_9ac6b5fa-b78b-4a95-a550-0621f217...,False,0,0,NaN
1,2024-11-27 02:32:25,209.78.40.60,a7429085-8825-43f1-be6e-c84d3c484537,Canada,GET,/promo-events.html,200,209.20,NaN,https://www.google.com,NaN,0.0,209.78.40.60_a7429085-8825-43f1-be6e-c84d3c484537,False,0,0,NaN
2,2024-11-27 03:15:40,15.54.111.208,475bbe1f-b25d-4af5-85c8-742ae8d3a301,USA,POST,/product/performance-analytics-tool/schedule-d...,408,382.25,Kago,https://www.linkedin.com,Performance Analytics Tool,0.0,15.54.111.208_475bbe1f-b25d-4af5-85c8-742ae8d3...,False,0,0,NaN
3,2024-11-27 03:06:03,87.218.51.15,0bd3fe9b-4b23-4f10-9ebc-c5032c868fe5,South Korea,GET,/index.html,200,388.18,NaN,https://www.facebook.com,NaN,0.0,87.218.51.15_0bd3fe9b-4b23-4f10-9ebc-c5032c868fe5,False,0,0,NaN
4,2024-11-27 03:09:03,151.19.182.101,fe5a0c1f-20fb-4bae-9758-e2e9170971b7,Canada,GET,/event.html,200,253.94,NaN,https://www.bing.com,NaN,0.0,151.19.182.101_fe5a0c1f-20fb-4bae-9758-e2e9170...,False,0,0,NaN


In [100]:
# Impute missing values
for column in df.columns:
    if df[column].dtype == 'object':  # Categorical columns (object type)
        df[column] = df[column].fillna('None')  # Impute with None
    else:  # Numerical columns
        df[column] = df[column].fillna(0)

In [101]:
# Fixing outliers in the Response Time column
df['Response Time (ms)'] = pd.to_numeric(df['Response Time (ms)'], errors='coerce')
df['Response Time (ms)'] = df['Response Time (ms)'].fillna(df['Response Time (ms)'].median())

Q1 = df['Response Time (ms)'].quantile(0.25)
Q3 = df['Response Time (ms)'].quantile(0.75)
IQR = Q3 - Q1

# Define bounds
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Calculate 95th percentile as cap value
cap_value = df['Response Time (ms)'].quantile(0.95)

# Replace outliers
df['Response Time (ms)'] = df['Response Time (ms)'].apply(
    lambda x: cap_value if x > upper_bound else (Q1 if x < lower_bound else x)
)

In [102]:
# Break up timestamp
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df['hour'] = df['Timestamp'].dt.hour
df['day_of_week'] = df['Timestamp'].dt.day_of_week
df['month'] = df['Timestamp'].dt.month
df['year'] = df['Timestamp'].dt.year
df['is_weekend'] = df['day_of_week'].apply(lambda x: 1 if x >= 5 else 0)

In [103]:
# Isolate referrer names
df['Referrer']= df['Referrer'].str.extract(r"https?://www\.([a-z]+)\.com", expand=False).fillna("direct")

In [104]:
# Rename Price to revenue
df["Revenue"]=df["Price"]


In [105]:
sales_cols = ['Sales Agent', 'Product']
for col in sales_cols:
    if df[col].isna().any():
        df[col]=df[col].fillna("None")

df['Revenue']=df['Revenue'].fillna(0)

In [106]:
def categorize_url(method, path):
    """Categorizes URLs based on method and path patterns"""
    path = str(path).lower()  # Ensure string and case-insensitive
    
    # Product Pages
    if '/product/' in path:
        if 'schedule-demo' in path:
            return "Demo Request"
        elif 'request.php' in path:
            return "Product Purchase"
        elif 'feedback.php' in path:
            return "Product Feedback"
        else:
            return "Product View"
    
    # Sales Actions
    elif any(p in path for p in ['/buy-', '/checkout', '/request-quote']):
        return "Sales Conversion"
    
    # Marketing
    elif any(p in path for p in ['/promo-', '/special-offers', '/newsletter']):
        return "Marketing Content"
    
    # Support
    elif any(p in path for p in ['/customer-support', '/faq', '/bug-tickets']):
        return "Support"
    
    # Company Info
    elif any(p in path for p in ['/about-us', '/contact-sales']):
        return "Company Info"
    
    # Static Assets
    elif any(ext in path for ext in ['.jpg', '.png', '.css', '.js', '/images/']):
        return "Static Asset"
    
    # Homepage
    elif path in ["/", "/index.html", "/home"]:
        return "Homepage"
    
    else:
        return "Other"

# Apply to DataFrame (assuming df has 'Method' and 'URL' columns)
df['Request Type'] = df.apply(
    lambda row: categorize_url(row['Method'], row['URL']), 
    axis=1
)

In [107]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125000 entries, 0 to 124999
Data columns (total 24 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   Timestamp                  125000 non-null  datetime64[ns]
 1   IP Address                 125000 non-null  object        
 2   Session ID                 125000 non-null  object        
 3   Country                    125000 non-null  object        
 4   Method                     125000 non-null  object        
 5   URL                        125000 non-null  object        
 6   Status Code                125000 non-null  int64         
 7   Response Time (ms)         125000 non-null  float64       
 8   Sales Agent                125000 non-null  object        
 9   Referrer                   125000 non-null  object        
 10  Product                    125000 non-null  object        
 11  Price                      125000 non-null  float64 

In [108]:
# Get unique IP addresses and their count
unique_ips = df["IP Address"].nunique()

# Count visits per IP (this won't count duplicates anymore)
ip_counts = df["IP Address"].value_counts()

# Map visit counts to the original DataFrame
df["visit_count"] = df["IP Address"].map(ip_counts)

# Classify visitor type (New or Returning)
df["visitor_type"] = df["visit_count"].apply(lambda x: "New" if x == 1 else "Returning")

# Count new and returning visitors
new_visitors = df[df["visitor_type"] == "New"]["IP Address"].nunique()
returning_visitors = df[df["visitor_type"] == "Returning"]["IP Address"].nunique()

# Print results
print("Total Unique IPs (counted only once):", unique_ips)
print("New Visitors:", new_visitors)
print("Returning Visitors:", returning_visitors)
print("Total Visits (All IPs):", ip_counts.sum())


Total Unique IPs (counted only once): 104136
New Visitors: 85784
Returning Visitors: 18352
Total Visits (All IPs): 125000


In [109]:
print("Unique Request Types:")
print(df["Request Type"].unique())


Unique Request Types:
['Company Info' 'Marketing Content' 'Demo Request' 'Homepage' 'Other'
 'Product Feedback' 'Support' 'Product View' 'Product Purchase'
 'Sales Conversion' 'Static Asset']


In [110]:
df.columns

Index(['Timestamp', 'IP Address', 'Session ID', 'Country', 'Method', 'URL',
       'Status Code', 'Response Time (ms)', 'Sales Agent', 'Referrer',
       'Product', 'Price', 'IP_Session', 'viewed_pricing_after_demo',
       'pages_after_demo', 'sessions_after_demo', 'time_to_purchase', 'hour',
       'day_of_week', 'month', 'year', 'is_weekend', 'Revenue', 'Request Type',
       'visit_count', 'visitor_type'],
      dtype='object')

In [ ]:
file='data/cleaned_logs/cleaned12.csv'
df.to_csv(file, index=False)

print(f"CSV file {file} has been saved successfully.")

CSV file data/cleaned_logs/cleaned11.csv has been saved successfully.
